In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [ ]:
# Importing packages
from pyspark.sql import SparkSession
import time

# Creating a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Reading in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

home_sales_revised_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=',', header=True)


In [ ]:
# Previewing the dataframe
home_sales_revised_df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [ ]:
 # Showing schema 
home_sales_revised_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [ ]:
 # Converting several column types of our dataframe using "selectExpr" 
home_sales_df = home_sales_revised_df.selectExpr("cast(date as timestamp) date",
    "cast(date_built as int) date_built",
    "cast(price as float) price",
    "cast(bedrooms as int) bedrooms",
    "cast(bathrooms as int) bathrooms",
    "cast(sqft_living as float) sqft_living",
    "cast(sqft_lot as float) sqft_lot",
    "cast(floors as int) floors",
    "cast(waterfront as binary) waterfront",
    "cast(view as float) view")

In [ ]:
# Checking that the above operation was successful
home_sales_df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: float (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: float (nullable = true)
 |-- sqft_lot: float (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: binary (nullable = true)
 |-- view: float (nullable = true)



In [ ]:
 # Importing date time functions
from pyspark.sql.functions import year

In [ ]:
 # Creating year as a new column using the "year" function and adding it onto the dataframe
home_sales_df = home_sales_df.withColumn('year', year(home_sales_df['date'])) 

home_sales_df.show(5)

+-------------------+----------+--------+--------+---------+-----------+--------+------+----------+----+----+
|               date|date_built|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+-------------------+----------+--------+--------+---------+-----------+--------+------+----------+----+----+
|2022-04-08 00:00:00|      2016|936923.0|       4|        3|     3167.0| 11733.0|     2|      [31]|76.0|2022|
|2021-06-13 00:00:00|      2013|379628.0|       2|        2|     2235.0| 14384.0|     1|      [30]|23.0|2021|
|2019-04-12 00:00:00|      2014|417866.0|       2|        2|     2127.0| 10575.0|     2|      [30]| 0.0|2019|
|2019-10-16 00:00:00|      2016|239895.0|       2|        2|     1631.0| 11149.0|     2|      [30]| 0.0|2019|
|2022-01-08 00:00:00|      2017|424418.0|       3|        2|     2249.0| 13878.0|     2|      [30]| 4.0|2022|
+-------------------+----------+--------+--------+---------+-----------+--------+------+----------+----+----+
only showi

In [ ]:
# 2. Creating a temporary view of the DataFrame.

home_sales_df.createOrReplaceTempView('home_sales')

In [ ]:
spark.sql("SELECT * FROM home_sales LIMIT 5").show()

+-------------------+----------+--------+--------+---------+-----------+--------+------+----------+----+----+
|               date|date_built|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+-------------------+----------+--------+--------+---------+-----------+--------+------+----------+----+----+
|2022-04-08 00:00:00|      2016|936923.0|       4|        3|     3167.0| 11733.0|     2|      [31]|76.0|2022|
|2021-06-13 00:00:00|      2013|379628.0|       2|        2|     2235.0| 14384.0|     1|      [30]|23.0|2021|
|2019-04-12 00:00:00|      2014|417866.0|       2|        2|     2127.0| 10575.0|     2|      [30]| 0.0|2019|
|2019-10-16 00:00:00|      2016|239895.0|       2|        2|     1631.0| 11149.0|     2|      [30]| 0.0|2019|
|2022-01-08 00:00:00|      2017|424418.0|       3|        2|     2249.0| 13878.0|     2|      [30]| 4.0|2022|
+-------------------+----------+--------+--------+---------+-----------+--------+------+----------+----+----+



In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
(SELECT round(avg(price),2) AS avg_price, year 
FROM home_sales
WHERE bedrooms == 4 
GROUP BY year)
"""
spark.sql(query).show()

+---------+----+
|avg_price|year|
+---------+----+
|296363.88|2022|
| 300263.7|2019|
|298353.78|2020|
|301819.44|2021|
+---------+----+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
(SELECT round(avg(price),2) AS avg_price, date_built, bedrooms, bathrooms
FROM home_sales
WHERE bedrooms == 3 AND bathrooms == 3
GROUP BY date_built, bedrooms, bathrooms
ORDER BY date_built)
"""
spark.sql(query).show()


+---------+----------+--------+---------+
|avg_price|date_built|bedrooms|bathrooms|
+---------+----------+--------+---------+
|292859.62|      2010|       3|        3|
|291117.47|      2011|       3|        3|
|293683.19|      2012|       3|        3|
|295962.27|      2013|       3|        3|
|290852.27|      2014|       3|        3|
| 288770.3|      2015|       3|        3|
|290555.07|      2016|       3|        3|
|292676.79|      2017|       3|        3|
+---------+----------+--------+---------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
(SELECT round(avg(price),2) AS avg_price, date_built, bedrooms, bathrooms, floors, avg(sqft_living) AS avg_sqft_living
FROM home_sales
WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000.0
GROUP BY date_built, bedrooms, bathrooms, floors
ORDER BY date_built)
"""
spark.sql(query).show()


+---------+----------+--------+---------+------+------------------+
|avg_price|date_built|bedrooms|bathrooms|floors|   avg_sqft_living|
+---------+----------+--------+---------+------+------------------+
|285010.22|      2010|       3|        3|     2|2264.4602272727275|
|276553.81|      2011|       3|        3|     2|2264.0994152046783|
|307539.97|      2012|       3|        3|     2|2334.5714285714284|
|303676.79|      2013|       3|        3|     2|        2296.36875|
|298264.72|      2014|       3|        3|     2| 2285.603448275862|
|297609.97|      2015|       3|        3|     2|2308.9251336898396|
| 293965.1|      2016|       3|        3|     2|2274.1744186046512|
|280317.58|      2017|       3|        3|     2|2274.0054945054944|
+---------+----------+--------+---------+------+------------------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
# a - we'll be creating a query first

query = """
(SELECT round(avg(view),2) AS avg_view, round(avg(price),2) AS avg_price
FROM home_sales
WHERE price >= 350000.0)
"""
spark.sql(query).show()
start_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

+--------+---------+
|avg_view|avg_price|
+--------+---------+
|   32.26|473796.26|
+--------+---------+

--- 0.0004718303680419922 seconds ---


In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
# b - we'll be inserting the query directly in the sql code

start_time = time.time()

spark.sql("""SELECT round(avg(view),2) AS avg_view, round(avg(price),2) AS avg_price FROM home_sales WHERE price >= 350000.0""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------+---------+
|avg_view|avg_price|
+--------+---------+
|   32.26|473796.26|
+--------+---------+

--- 0.8326950073242188 seconds ---


In [ ]:
# 7. Caching the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [ ]:
# 8. Checking if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""SELECT round(avg(view),2) AS avg_view, round(avg(price),2) AS avg_price FROM home_sales WHERE price >= 350000.0""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------+---------+
|avg_view|avg_price|
+--------+---------+
|   32.26|473796.26|
+--------+---------+

--- 0.2773871421813965 seconds ---


In [ ]:
# 10. Partitioning by the "date_built" field on the formatted parquet home sales data 
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [ ]:
# 11. Reading the parquet formatted data.
p_df_p=spark.read.parquet('home_sales_partitioned')

In [ ]:
# 12. Creating a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_home_sales_p')

In [ ]:
# 13. Running the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determining the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""SELECT round(avg(view),2) AS avg_view, round(avg(price),2) AS avg_price FROM p_home_sales_p WHERE price >= 350000.0""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------+---------+
|avg_view|avg_price|
+--------+---------+
|   32.26|473796.26|
+--------+---------+

--- 0.4460928440093994 seconds ---


In [ ]:
# 14. Uncaching the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [ ]:
# 15. Checking if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False